In [10]:
import pandas
import spotipy
import redis
from spotipy.oauth2 import SpotifyClientCredentials
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
# Set spotipy object
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id='118aa19f2b66476fbc062f0ac146d8b5',
    client_secret='7ca95a3159ab4391bee70f70d47a9271'
))

In [3]:
# Connetto a redis e lancio un ping
REDIS_HOST = "jupyter_redis"
REDIS_PORT = 6379
REDIS = redis.Redis(host=REDIS_HOST, port=REDIS_PORT)
# test the connection
REDIS.ping()

True

# Prove

In [9]:
sp.audio_features('1239722322')

[None]

In [7]:
print(REDIS.get('wdfga')==None)

True


### Connessione POSTGRES da spark

In [11]:
from pyspark.sql.functions import to_timestamp, date_format
import pyspark.sql.functions as f

spark = SparkSession \
    .builder \
    .appName('DBAnalysis') \
    .config('spark.driver.extraClassPath', 'postgresql-42.2.10.jar') \
    .getOrCreate()

In [12]:
properties = {
    'driver': 'org.postgresql.Driver',
    'url': 'jdbc:postgresql://postgres:5432/postgres',
    'user': 'postgres',
    'password': 'postgres1234',
    'dbtable': ' spotify_details',
}
properties_dwh = {
    'driver': 'org.postgresql.Driver',
    'url': 'jdbc:postgresql://postgres_dwh:5432/postgres',
    'user': 'postgres',
    'password': 'postgres1234',
    'dbtable': ' spotify_details',
}

In [13]:
df2 = spark.read \
    .format('jdbc') \
    .option('driver', properties['driver']) \
    .option('url', properties['url']) \
    .option('user', properties['user']) \
    .option('password', properties['password']) \
    .option('dbtable', properties['dbtable']) \
    .load()

In [14]:
df2.printSchema()

root
 |-- id_playlist: string (nullable = true)
 |-- name_playlist: string (nullable = true)
 |-- id_track: string (nullable = true)
 |-- name_track: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- time_signature: string (nullable = true)



In [30]:
# Creo la colonna year_month
df3 = df2.withColumn('year_month', date_format(
        to_timestamp(df2.timestamp, "yyyy-MM-dd'T'HH:mm:ssXXX"), 
        "yyyy-MM"
    ).alias('year_month')) 

In [31]:
df3.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+
|         id_playlist|       name_playlist|            id_track|          name_track|           timestamp|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|year_month|
+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+----------+
|7I5p2wl30OGnqmgrA...|      COVID Sessions|5sXPAGXZEnl7dMReM...|       Cloud Trippin|2020-09-16T21:46:15Z|       0.796| 0.419|  2|  -9.686|   1|     0.0704|       0.114|           0.938|   0.105|  0.165| 94.958|     205547|             4|   2020-09|


In [32]:
# Aggrego sulla playlist e conto le ricorrenze
df4 = df3.groupby('id_playlist', 'year_month').count().sort('id_playlist', ascending=False)
df4.show(10)

+--------------------+----------+-----+
|         id_playlist|year_month|count|
+--------------------+----------+-----+
|7zvpNOLMErZqvGA4m...|   2020-08|   22|
|7zvpNOLMErZqvGA4m...|   2020-12|   12|
|7zvpNOLMErZqvGA4m...|   2021-01|    1|
|7zvpNOLMErZqvGA4m...|   2020-11|   19|
|7zvpNOLMErZqvGA4m...|   2020-10|   35|
|7zvpNOLMErZqvGA4m...|   2020-09|    2|
|7z3q0iLySqsrCJ0yu...|   2021-01|    2|
|7z3q0iLySqsrCJ0yu...|   2020-09|    3|
|7z3q0iLySqsrCJ0yu...|   2020-12|    3|
|7z3q0iLySqsrCJ0yu...|   2020-11|    7|
+--------------------+----------+-----+
only showing top 10 rows



In [33]:
df2.groupby('id_playlist')\
.count()\
.sort('id_playlist', ascending=False)\
.show()


+--------------------+-----+
|         id_playlist|count|
+--------------------+-----+
|7zvpNOLMErZqvGA4m...|   91|
|7z3q0iLySqsrCJ0yu...|   19|
|7z3hyKsoXNxKSYUCe...|   52|
|7ymbfYgGdOqRns9yE...|   79|
|7ygraCBBpUscXNrAc...|   56|
|7yV3JkSqKv3GVTqmD...|  145|
|7y87QYIUUmUGXJpMC...|   28|
|7xiMMVwvFjqjMZ6qX...|   61|
|7xO13RrgA0UIyysNf...|   96|
|7xJkD9gWn8u0MnOqn...|   85|
|7xFIxOjlXq2nFNGex...|  201|
|7w1y3shCr8uDx1aKR...|  101|
|7vhibs6g5WZL3PHKL...|  190|
|7v9h1AMd65L8MCQaJ...|  271|
|7ubkVtDuMGueTCxSy...|   28|
|7uSDpfjUmrtjbv422...|   81|
|7uDicALsURHtRq8lO...|   74|
|7tqmR5oz5ApHFnS6f...|  690|
|7tZeMUzIOBM6H1m7J...|   54|
|7tZ8MM7p8F3cgyPPr...| 1485|
+--------------------+-----+
only showing top 20 rows



In [34]:
# Per assegnare un mese alla playlist si é deciso di scegliere il mese con maggiori "aggiunte" di canzoni
# Per ogni playlist seleziono solo quella con ricorrenze per mese maggiore 
from pyspark.sql import Window

w = Window.partitionBy('id_playlist')

df5 = df4.withColumn('max', f.max('count').over(w))\
    .where(f.col('count') == f.col('max'))\
    .drop('max', 'count')

df5.sort('id_playlist', ascending=False)\
    .show()

+--------------------+----------+
|         id_playlist|year_month|
+--------------------+----------+
|7zvpNOLMErZqvGA4m...|   2020-10|
|7z3q0iLySqsrCJ0yu...|   2020-11|
|7z3hyKsoXNxKSYUCe...|   2020-12|
|7ymbfYgGdOqRns9yE...|   2020-04|
|7ygraCBBpUscXNrAc...|   2020-09|
|7yV3JkSqKv3GVTqmD...|   2020-04|
|7y87QYIUUmUGXJpMC...|   2020-12|
|7xiMMVwvFjqjMZ6qX...|   2020-11|
|7xO13RrgA0UIyysNf...|   2020-03|
|7xJkD9gWn8u0MnOqn...|   2019-12|
|7xFIxOjlXq2nFNGex...|   2019-01|
|7w1y3shCr8uDx1aKR...|   2020-03|
|7vhibs6g5WZL3PHKL...|   2020-06|
|7v9h1AMd65L8MCQaJ...|   2020-03|
|7ubkVtDuMGueTCxSy...|   2021-01|
|7uSDpfjUmrtjbv422...|   2020-07|
|7uDicALsURHtRq8lO...|   2020-05|
|7tqmR5oz5ApHFnS6f...|   2020-05|
|7tZeMUzIOBM6H1m7J...|   2020-11|
|7tZ8MM7p8F3cgyPPr...|   2020-05|
+--------------------+----------+
only showing top 20 rows



In [35]:
df5\
.where(f.col('year_month')>"2020-00")\
.groupby('year_month')\
.count()\
.sort('year_month', ascending=False)\
.show(100)

+----------+-----+
|year_month|count|
+----------+-----+
|   2021-01|  163|
|   2020-12|  398|
|   2020-11|  245|
|   2020-10|  158|
|   2020-09|  108|
|   2020-08|  128|
|   2020-07|  103|
|   2020-06|   87|
|   2020-05|  140|
|   2020-04|  199|
|   2020-03|  165|
|   2020-02|   13|
|   2020-01|   13|
+----------+-----+



In [36]:
# Il df completo ha l'informazione sul mese di riferimento assegnata ad ogni playlist
df_complete = df2.join(df5, on=['id_playlist'], how='left')

df_complete\
.select('id_playlist', 'name_playlist','year_month')\
.show(10)

+--------------------+-------------+----------+
|         id_playlist|name_playlist|year_month|
+--------------------+-------------+----------+
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
+--------------------+-------------+----------+
only showing top 10 rows



In [45]:
df_complete2 =  df_complete.groupBy("id_playlist", 'name_playlist' , 'year_month')\
.agg(f.mean('danceability'),f.stddev_pop('danceability'),f.mean('energy'),f.stddev_pop('energy'),f.mean('valence'),f.stddev_pop('valence'))\
.sort('year_month', ascending=True)

newColumns = ["id_playlist","name_playlist","year_month","avgdanceability","stdddanceability","avgenergy","stddenergy","avgvalence","stddvalence"]
df_complete2 = df_complete2.toDF(*newColumns)


In [49]:
df_complete3 = df_complete2.groupBy('year_month')\
.agg(f.mean('avgdanceability'),f.mean('stdddanceability'),f.mean('avgenergy'),f.mean('stddenergy'),f.mean('avgvalence'),f.mean('stddvalence'))\
.sort('year_month', ascending=True)

newColumns = ["timestamp","mean_danceability","stdev_danceability","mean_energy","stdev_energy","mean_valence","stdev_valence"]
df_complete3 = df_complete3.toDF(*newColumns)
df_complete3.show()

+---------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|timestamp|  mean_danceability| stdev_danceability|        mean_energy|       stdev_energy|       mean_valence|      stdev_valence|
+---------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|  2014-08|  0.701656417669636|0.12133841745357912| 0.6611926248695081| 0.1479234495250273| 0.5983750379057203|0.20364093027973523|
|  2015-02| 0.7312476190476187|0.10270714547891151| 0.8444642857142856| 0.1259911794369684| 0.8984500000000004|0.11377134493611045|
|  2015-09|0.48336964980544794| 0.1729158933139975|0.46534852140077837|0.23220261583566745| 0.5115595330739297| 0.2595165712165001|
|  2016-02| 0.6928454208575969| 0.1361570280810571| 0.6586084700899962|0.16726858132286893| 0.5215991529909993|0.22728163432525458|
|  2016-07| 0.7658982558139534|0.10765442449447626| 0.7338982558139536|0.110

In [41]:
properties_dwh = {
    'driver': 'org.postgresql.Driver',
    'url': 'jdbc:postgresql://postgres_dwh:5432/postgres',
    'user': 'postgres',
    'password': 'postgres1234'
}

In [42]:
#Save the dataframe to the table. 
df_complete2.write.jdbc(url=properties_dwh['url'],table='audiofeatures_stat',mode='overwrite',properties=properties_dwh)

In [38]:
df_complete2.select('avg(danceability)','stddev_pop(danceability)').show(1000)

+-------------------+------------------------+
|  avg(danceability)|stddev_pop(danceability)|
+-------------------+------------------------+
| 0.7523764019739783|     0.10029577881212025|
| 0.6509364333652937|       0.142381056095038|
| 0.7312476190476187|     0.10270714547891151|
|0.48336964980544794|      0.1729158933139975|
| 0.6928454208575969|      0.1361570280810571|
| 0.7658982558139534|     0.10765442449447626|
| 0.6440680786686838|     0.13315948791988527|
| 0.5216804123711342|     0.15540457429736335|
| 0.6845075376884431|     0.10029804175238993|
| 0.5525405405405405|     0.15215353695170886|
| 0.4438835341365461|     0.10682804601612038|
| 0.5859059602649006|      0.1244849113076891|
| 0.7593415637860079|     0.13087305208546038|
|  0.586126315789474|     0.12702927487748744|
|  0.620443298969072|     0.13348994858273777|
| 0.7321872483221483|     0.09598771159368279|
| 0.7550560224089636|      0.0830796061083919|
|  0.648693825042881|     0.12755909973080737|
| 0.636631782

In [25]:
df_complete2.select('avg(valence)','stddev_pop(valence)').show(1000)

+-------------------+-------------------+
|       avg(valence)|stddev_pop(valence)|
+-------------------+-------------------+
|  0.486153978906999|0.23893464502551406|
| 0.7105960969044416|0.16834721553395637|
| 0.8984500000000004|0.11377134493611045|
| 0.5115595330739297| 0.2595165712165001|
| 0.5215991529909993|0.22728163432525458|
| 0.6196502906976739|0.17537279909582962|
| 0.7046030257186072| 0.1887689098399449|
| 0.5328670103092779|0.26047999971569114|
| 0.5482718592964828| 0.1957200408337894|
| 0.6057144144144145|0.20608776309749924|
|0.19954979919678725|0.12620733121244146|
| 0.2723017218543046|0.17914105074845466|
| 0.5451876543209878|0.21438383237597403|
|0.47321000000000013|0.22161434747160025|
| 0.7097522147650998|0.19593323842042115|
| 0.5074216494845362|0.21505597843630556|
| 0.6354621848739491| 0.1784984581177338|
|  0.514698799313894|0.22140816153684997|
|  0.743748062015504|0.19924126063168912|
| 0.6642810298102984|0.18190726774231994|
|0.46214112149532705| 0.2255386267

In [26]:
df_complete2.select('avg(energy)','stddev_pop(energy)').show(1000)
# Dai risultati vediamo come le playlist non sono poi cosí variabili
# Probabilmente per il concetto stesso di playlist

+-------------------+-------------------+
|        avg(energy)| stddev_pop(energy)|
+-------------------+-------------------+
| 0.5944983849259768|0.14772635444699458|
| 0.7278868648130394|0.14812054460306004|
| 0.8444642857142856| 0.1259911794369684|
|0.46534852140077837|0.23220261583566745|
| 0.6586084700899962|0.16726858132286893|
| 0.7338982558139536|0.11002747329952982|
| 0.5906691376701966|0.19020124914047426|
| 0.4138025429553265|0.22626164805632054|
| 0.8025326633165832|0.10240935537726102|
| 0.6583513513513514|0.24250147760852506|
| 0.9134979919678715|0.07271057907361775|
| 0.6631522633744856|0.13726336714468906|
| 0.8932105960264901|0.09118360427757134|
| 0.6109363157894737|0.20099070991127213|
| 0.7521758389261752|0.12632940304600762|
| 0.6422783505154641|0.17710901426342385|
| 0.7259887955182068|0.10506212588096464|
| 0.7531493996569478| 0.1411220388817074|
| 0.6284108527131783|0.17066541510577468|
| 0.7371924119241192|0.09916299219472893|
|  0.635570093457944|0.14522924914

In [28]:
df_complete2.filter(df_complete2.year_month < '2020-01').count()

89

# Visualization


In [51]:
# Importing necessary Packages
import pandas as pd
import numpy as np
import time
import matplotlib.ticker as ticker
from collections import OrderedDict

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly import tools
import plotly.graph_objects as go

import psycopg2 # python package for Postgres

# Postgres settings
conn = psycopg2.connect(
    host = "postgres_dwh",
    port = "5432",
    dbname = "postgres",
    user = "postgres",
    password = "postgres1234"
)

In [52]:
# Postgres settings
conn = psycopg2.connect(
    host = "postgres_dwh",
    port = "5432",
    dbname = "postgres",
    user = "postgres",
    password = "postgres1234"
)

In [56]:
sql = """SELECT * FROM audiofeatures_stat ORDER BY timestamp DESC """
df = pd.read_sql(sql, conn)
df

,timestamp,mean_danceability,stdev_danceability,mean_energy,stdev_energy,mean_valence,stdev_valence
0,2021-01,0.657776,0.126055,0.634363,0.156073,0.519023,0.209554
1,2020-12,0.665344,0.127575,0.653886,0.155309,0.529767,0.210282
2,2020-11,0.671206,0.124667,0.643096,0.152639,0.519460,0.209880
3,2020-10,0.667532,0.126117,0.643718,0.151551,0.520319,0.210346
4,2020-09,0.676567,0.124426,0.649350,0.152928,0.535423,0.210585
5,2020-08,0.677097,0.122269,0.663464,0.145930,0.557456,0.202896
6,2020-07,0.679285,0.124324,0.665308,0.154901,0.552449,0.211853
7,2020-06,0.669414,0.125374,0.672611,0.152110,0.545875,0.209474
8,2020-05,0.663394,0.127019,0.643243,0.157875,0.536243,0.213068
9,2020-04,0.657779,0.126787,0.652174,0.158452,0.520629,0.208267


In [63]:
import plotly.express as px

fig = px.line(df, x="timestamp", y=["mean_danceability", "mean_energy", "mean_valence"])
fig.show()

fig2 = px.line(df, x="timestamp", y=["stdev_danceability", "stdev_energy", "stdev_valence"])
fig2.show()

In [65]:
sql = """SELECT * FROM month_distribution ORDER BY year_month DESC """
df2 = pd.read_sql(sql, conn)
df2

,year_month,count
0,2021-01,163
1,2020-12,398
2,2020-11,245
3,2020-10,158
4,2020-09,108
5,2020-08,128
6,2020-07,103
7,2020-06,87
8,2020-05,140
9,2020-04,199


In [66]:
fig = px.bar(df2, x='year_month', y='count')
fig.show()

In [67]:
sql = """SELECT * FROM count_tracks_distribution ORDER BY count DESC """
df3 = pd.read_sql(sql, conn)
df3

,id_playlist,count
0,2GIlRktSPxYkogb7ctmgKH,9994
1,7hfD9vEDz8G9YG68bBlxzH,6546
2,0vhKawH7w2r03BOsaqrED0,6125
3,6TggnksNynkCIjwSAR4K56,5809
4,5VI1X6o7b3qQh9K29ZOw32,3706
...,...,...
1956,5PdsQkknlrXwNTEO89ihMQ,3
1957,1Vohgy6NK41X0xVYjC9tV1,1
1958,5Q5H1utz8JAC8iFfeZSZdb,1
1959,5n1vcNfSDxnMJO1C9s1nRK,1


In [75]:
df3['0-50'] = np.where((df3['count'].between(0,50)), True, False)
df3['51-100'] = np.where((df3['count'].between(51,100)), True, False)
df3['101-200'] = np.where((df3['count'].between(101,200)), True, False)
df3['201-500'] = np.where((df3['count'].between(201,500)), True, False)
df3['501-1000'] = np.where((df3['count'].between(501,1000)), True, False)
df3['1001+'] = np.where(df3['count']>1000, True, False)

In [76]:
df3

,id_playlist,count,0-50,51-100,101-200,201-500,501-1000,1001+
0,2GIlRktSPxYkogb7ctmgKH,9994,False,False,False,False,False,True
1,7hfD9vEDz8G9YG68bBlxzH,6546,False,False,False,False,False,True
2,0vhKawH7w2r03BOsaqrED0,6125,False,False,False,False,False,True
3,6TggnksNynkCIjwSAR4K56,5809,False,False,False,False,False,True
4,5VI1X6o7b3qQh9K29ZOw32,3706,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...
1956,5PdsQkknlrXwNTEO89ihMQ,3,True,False,False,False,False,False
1957,1Vohgy6NK41X0xVYjC9tV1,1,True,False,False,False,False,False
1958,5Q5H1utz8JAC8iFfeZSZdb,1,True,False,False,False,False,False
1959,5n1vcNfSDxnMJO1C9s1nRK,1,True,False,False,False,False,False


In [91]:
columns=['0-50','51-100','101-200','201-500','501-1000','1001+']
lenght = []
for item in columns:
    lenght.append(len(df3[df3[item] == True]))
 
print(lenght)
counts = pd.DataFrame(data = lenght, index=columns, columns=["count"])
counts

[390, 514, 544, 385, 90, 38]


,count
0-50,390
51-100,514
101-200,544
201-500,385
501-1000,90
1001+,38


In [93]:
fig = px.bar(counts, y='count')
fig.show()

In [94]:
# Importing necessary Packages
import pandas as pd
import numpy as np
import time
import matplotlib.ticker as ticker
from collections import OrderedDict

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly import tools
import plotly.graph_objects as go

import psycopg2 # python package for Postgres

# Postgres settings
conn = psycopg2.connect(
    host = "postgres_dwh",
    port = "5432",
    dbname = "postgres",
    user = "postgres",
    password = "postgres1234"
)

sql = """SELECT * FROM count_tracks_distribution ORDER BY count DESC """
df3 = pd.read_sql(sql, conn)
df3

df3['0-50'] = np.where((df3['count'].between(0,50)), True, False)
df3['51-100'] = np.where((df3['count'].between(51,100)), True, False)
df3['101-200'] = np.where((df3['count'].between(101,200)), True, False)
df3['201-500'] = np.where((df3['count'].between(201,500)), True, False)
df3['501-1000'] = np.where((df3['count'].between(501,1000)), True, False)
df3['1001+'] = np.where(df3['count']>1000, True, False)
columns=['0-50','51-100','101-200','201-500','501-1000','1001+']
lenght = []
for item in columns:
    lenght.append(len(df3[df3[item] == True]))

counts = pd.DataFrame(data = lenght, index=columns, columns=["count"])


#cose dash

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

fig = px.bar(counts, y='count')

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

